# Installing PySpark on Google Colab

Special thanks to my colleagues Jeff & James for content in this notebook <3

In [ ]:
!apt update

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

# Resilient Distributed Datasets

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it.

Use an RDD when:
[(quoted from databricks)](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html)

- you want low-level transformation and actions and control on your dataset;
- your data is unstructured, such as media streams or streams of text;
- you want to manipulate your data with functional programming constructs than domain specific expressions;
- you don’t care about imposing a schema, such as columnar format, while processing or accessing data attributes by name or column

RDDs have 2 operations that you can perform on them:
- Transformations (create a new RDD)
- Actions (return results)

Note: transformations are lazy operators, they won't actually perform the transformation until an action is performed.

In [ ]:
import pyspark

In [ ]:
# create a new RDD 
nums = list(range(1,1001))

sc = pyspark.SparkContext('local[*]')

rdd = sc.parallelize(nums, numSlices=10)
rdd.getNumPartitions()

#### Examples of Actions


In [ ]:
# first
rdd.first()

In [ ]:
# take
rdd.take(10)

In [ ]:
# collect
rdd.collect()

In [ ]:
# grab first partition using glom
rdd.glom().collect()[3]

In [ ]:
print(type(rdd))

#### Examples of Transformations
- map
- filter 

In [ ]:
# map
# use a lambda to return x+1 if x is even, else just return x
even_rdd = rdd.map(lambda x: x + 1 if x % 2 == 0 else x)

In [ ]:
even_rdd.take(5)

In [ ]:
# now let's try to just return even results
rdd.map(lambda x: x if x % 2 == 0)
# can't really use a map for this...

In [ ]:
# try with filter now
only_evens = rdd.filter(lambda x: x % 2 == 0)
only_evens.take(10)

In [ ]:
# stop your pyspark context instance
# can't have multiple connections at once!
sc.stop()

# Spark DataFrame

Dataframes in PySpark are the distributed collection of structured or semi-structured data. This data in Dataframe is stored in rows under named columns which is similar to the relational database tables or excel sheets. 

Use a Dataframe when:
[(also quoted from databricks)](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html)

- you want rich semantics, high-level abstractions, and domain specific APIs, use DataFrame
- your processing demands high-level expressions, filters, maps, aggregation, averages, sum, SQL queries, columnar access and use of lambda functions on semi-structured data, use DataFrame
- you want higher degree of type-safety at compile time, want typed JVM objects, take advantage of Catalyst optimization, and benefit from Tungsten’s efficient code generation, use Dataset;
- you want unification and simplification of APIs across Spark Libraries, use DataFrame or Dataset;
- If you are a R user, use DataFrames.
- If you are a Python user, use DataFrames and resort back to RDDs if you need more control.

Note: Machine learning algorithms are run on DataFrames

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master('local').getOrCreate()

Need to upload data to access here!

Link: https://www.kaggle.com/usdot/flight-delays

In [ ]:
# reading in pyspark df - check file location after you upload it!
spark_df = spark.read.csv('/flights.csv', header='true', inferSchema='true')

# observing the datatype of df
type(spark_df)

In [ ]:
# number of rows
print(spark_df.count())

In [ ]:
# number of columns
print(len(spark_df.columns))

In [ ]:
# check first five rows v1
spark_df.take(5)

In [ ]:
# check first five rows v2
spark_df.head(5)

In [ ]:
# check column datatypes
spark_df.printSchema()

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

# check for nans in each column
spark_df.select([count(when(isnan(c), c)).alias(c) for c in spark_df.columns]).show()

In [ ]:
# but NOT the same as nulls!
spark_df.select([count(when(col(c).isNull(), c)).alias(c) for c in spark_df.columns]).show()

In [ ]:
# can groupby
spark_df.groupby('DAY_OF_WEEK').count().show()

In [ ]:
# only want certain columns
spark_df = spark_df.select(col("MONTH"),col("DAY_OF_WEEK"), col("AIR_TIME"), col('ARRIVAL_DELAY'))

In [ ]:
spark_df.columns

In [ ]:
spark_df.take(5)

In [ ]:
# need to drop nulls in those columns
spark_df = spark_df.na.drop(subset=["AIR_TIME", "ARRIVAL_DELAY"])

Now - time to prep our target! Going to predict whether there was a delay in arrival or not

In [ ]:
def prep_target(delay_value):
  if delay_value < 0: 
    return 0
  else: 
    return 1

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
# here, creating a User Defined Function, resulting in a boolean column
udfTargetToCategory = udf(prep_target, IntegerType())

preprocessed_df = spark_df.withColumn("delay_ind", udfTargetToCategory("ARRIVAL_DELAY"))


In [ ]:
preprocessed_df.take(5)

Need to encode!

https://spark.apache.org/docs/2.3.0/api/python/pyspark.ml.html#pyspark.ml.feature.OneHotEncoderEstimator

In [ ]:
from pyspark.ml import feature

ohe = feature.OneHotEncoderEstimator(inputCols=['MONTH', 'DAY_OF_WEEK'], 
                                     outputCols=['month_vec', 'day_vec'])
ohe_hot_encoded = ohe.fit(preprocessed_df).transform(preprocessed_df)
ohe_hot_encoded.head()

In [ ]:
ohe_hot_encoded.take(5)

In [ ]:
# only using a few of the features as inputs
features = ['AIR_TIME', 'month_vec', 'day_vec']
target = 'delay_ind'

# need to vectorize the inputs
vector = feature.VectorAssembler(inputCols = features, outputCol = 'features')
vectorized_df = vector.transform(ohe_hot_encoded)

In [ ]:
print(type(vector))

In [ ]:
vectorized_df.columns

In [ ]:
vectorized_df.take(5)

It appears I'm not alone in having trouble with `randomSplit`: https://medium.com/udemy-engineering/pyspark-under-the-hood-randomsplit-and-sample-inconsistencies-examined-7c6ec62644bc

(I'll note, though, that my problems were not rooted in this issue!)

In [ ]:
# train test split
# Implementing the solution discussed above

persist_df = vectorized_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

train, test = persist_df.randomSplit([0.7, 0.3], seed = 11)

In [ ]:
type(train)

In [ ]:
train.take(5)

In [ ]:
# Now let's try a classifier!
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'delay_ind', 
                            maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)

In [ ]:
 # Evaluate!
 from pyspark.ml.evaluation import BinaryClassificationEvaluator

 evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'prediction',
                                           labelCol = 'delay_ind')
 evaluator.evaluate(predictions) # Note: ROC/AUC score

#### Evaluate! How'd we do? Why?

- .5 ROC/AUC??? Guess: is our model only predicting a single class?


In [ ]:
# Explore our predictions
predictions.groupby('prediction').count().show()
# note - this is the size of the test set

In [ ]:
# Explore our original data
test.groupby('delay_ind').count().show()

In [ ]:
preprocessed_df.groupby('delay_ind').count().show()

Yup - it is only predicting the majority class.

How can we fix this? Let's try undersampling our majority so our classes are more balanced.

In [ ]:
# Note that these don't use square brackets!
major_df = preprocessed_df.filter(col('delay_ind') == 0)
minor_df = preprocessed_df.filter(col('delay_ind') == 1)

In [ ]:
# Down-sample - without replacement
sampled_majority_df = major_df.sample(False, .65)

In [ ]:
sampled_majority_df.count()

In [ ]:
combined_df = sampled_majority_df.unionAll(minor_df)

In [ ]:
combined_df.groupby('delay_ind').count().show()

In [ ]:
combined_df.columns

In [ ]:
ohe = feature.OneHotEncoderEstimator(inputCols=['MONTH', 'DAY_OF_WEEK'], 
                                     outputCols=['month_vec', 'day_vec'])
ohe_hot_encoded = ohe.fit(combined_df).transform(combined_df)
ohe_hot_encoded.head()

In [ ]:
features = ['AIR_TIME', 'month_vec', 'day_vec']
target = 'delay_ind'

vector = feature.VectorAssembler(inputCols = features, outputCol = 'features')
vectorized_df = vector.transform(ohe_hot_encoded)

In [ ]:
persist_df = vectorized_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

train, test = persist_df.randomSplit([0.7, 0.3], seed = 11)

In [ ]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'delay_ind', 
                            maxDepth = 9)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)

In [ ]:
 evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'prediction',
                                           labelCol = 'delay_ind')
 evaluator.evaluate(predictions)

#### Evaluate!

- 


# Other Notes

## Hadoop vs Spark: which is better?

> Spark has been found to run 100 times faster in-memory, and 10 times faster on disk. It’s also been used to sort 100 TB of data 3 times faster than Hadoop MapReduce on one-tenth of the machines. Spark has particularly been found to be faster on machine learning applications, such as Naive Bayes and k-means. Spark performance, as measured by processing speed, has been found to be optimal over Hadoop, for several reasons:
- Spark is not bound by input-output concerns every time it runs a selected part of a MapReduce task. It’s proven to be much faster for applications.
- Spark’s DAGs enable optimizations between steps. Hadoop doesn’t have any cyclical connection between MapReduce steps, meaning no performance tuning can occur at that level. However, if Spark is running on YARN with other shared services, performance might degrade and cause RAM overhead memory leaks. For this reason, if a user has a use-case of batch processing, Hadoop has been found to be the more efficient system.

Using Hadoop and Spark together
> There are several instances where you would want to use the two tools together. Despite some asking if Spark will replace Hadoop entirely because of the former’s processing power, they are meant to complement each other rather than compete